Beaching notebook

This notebook is going to explore the beaching behaviour of particles in the 2D coral flow model. We want to know what the flowfields, particle position and interpolation look like the moment before particles are beached. We can use any ParticleSet output.

First let's import the modules and define the outputfile we will analyse

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import xarray as xr
import cmocean
import matplotlib.animation as animation

name = 'output-corals-regridded'
fb = 'forward' #variable to determine whether the flowfields are analysed 'forward' or 'backward' in time
runtime=timedelta(seconds=10)                            # total time to execute the particleset
outputdt = timedelta(seconds=0.01)
filename = name+'.nc'
dfilename = name+fb+str(runtime.seconds)+'.nc'

The particle data and the flow data are in separate files

In [3]:
data = xr.open_dataset(dfilename)

flowdata = xr.open_dataset(filename)

Now that the data and flowfields are in the xarrray, we want to select only the beached particles

In [4]:
bdata = data.where(data['finaldistance'][:,-1]==0,drop=True)

Next we want to select only one trajectory. We are going to find the first moment at which the particle is beached so we can plot the trajectory at the timesteps leading up to that moment

In [5]:
traj = 5
one = bdata.isel(traj=traj)
onetraj = one.where(one['finaldistance']==0,drop=True)

In [6]:
divider = 1000*outputdt.microseconds                                # nanoseconds to outputdt
tb = round(float(onetraj.isel(obs=1)['time'].values)/divider)       # time of first beached moment
TB = round(float(onetraj.isel(obs=1)['time'].values)/1000000000)    # nanoseconds to seconds
print(TB)
print(tb)

2
164


Now we are going to create meshgrids to show the coral and flow gridpoints

In [7]:
x,y = np.meshgrid(flowdata['X'],flowdata['Y'])
um = np.ma.masked_invalid(flowdata['U'][0,:,:])                    # retrieve mask from flowfield to take out points over coral objects

lons = np.ma.masked_array(x,mask=um.mask)             # mask points in meshgrid
lons = lons.flatten()
ds = np.ma.masked_array(y,mask=um.mask)                 # mask points in meshgrid
ds = ds.flatten()

usm = np.invert(um.mask)
lonm = np.ma.masked_array(x,mask=usm)             # mask points in meshgrid
lonm = lonm.flatten()
dm = np.ma.masked_array(y,mask=usm)                 # mask points in meshgrid
dm = dm.flatten()

Plot the coral gridpoints, flow gridpoints, flow quiver and beaching trajectory of the selected particle

In [7]:
%matplotlib qt
fig = plt.figure(figsize=(13,10))
ax = plt.axes()
ax.plot(one['lon'].isel(obs=slice(tb-40,tb+1)),-one['z'].isel(obs=slice(tb-40,tb+1)),c='r',marker='o',label='particle trajectory')
ax.scatter(lons,ds,s=7,c='c',label='flow gridpoint')
ax.scatter(lonm,dm,s=15,c='k',label='coral gridpoint')
C = np.hypot(flowdata['U'].isel(T=TB),flowdata['V'].isel(T=TB))
ax.quiver(flowdata['X'],flowdata['Y'],flowdata['U'].isel(T=TB),flowdata['V'].isel(T=TB),C,angles='xy',scale=50)
plt.ylim(-one['z'].isel(obs=tb)-0.05,-one['z'].isel(obs=tb)+0.05)
plt.xlim(one['lon'].isel(obs=tb)-0.05,one['lon'].isel(obs=tb)+0.05)
plt.ylabel('Height [m]')
plt.xlabel('Horizontal distance [m]')
plt.title('Beaching trajectory '+str(traj))
plt.legend()
# plt.savefig('Figures/beachingTrajectory'+str(traj))

Here we look at some statistics of beaching

In [8]:
ct = np.zeros(len(data['finaldistance'][0]))
ct[-1] = np.count_nonzero(data['finaldistance'][:,-1]==0)
beac = np.zeros((int(ct[-1])),dtype=np.int64)
# beac[:,1] = np.nonzero(data['finaldistance'][:,-1]==0)[0]
bn = data['finaldistance'].where(data['finaldistance'][:,-1]==0,drop=True).values
for i in range(len(bn)):
    cn = 0
    for j in np.arange(len(bn[i])-2,-1,-1):
        if bn[i,j] == 0:
            cn += 1
        else:
            break
    beac[i] = 44 - cn/10
for i in np.arange(len(data['finaldistance'][0])-2,-1,-1): # each timestep
    ct[i] = np.count_nonzero(data['finaldistance'][:,i]==0) #count how many values are zero 

In [10]:
plt.hist(beac)
plt.title('Particles beached per timebin')
plt.ylabel('number of particles')
plt.xlabel('time beached [s]')
plt.ylim(bottom=0)

(0, 768.6)

In [8]:
fig = plt.figure(figsize=(18,2))
ax = plt.axes()
ax.set_facecolor('k')
cf = ax.contourf(flowdata['X'],flowdata['Y'],flowdata['U'][8,:,:],np.linspace(-10,-9,21),extend='both',cmap=cmocean.cm.dense_r)
sc = ax.scatter(bdata['lon'][:,0], -bdata['z'][:,0], c=bdata['z'][:,0],s=10, marker="o",cmap='Spectral_r')
topleft = 0.05
if fb == 'backward':
    topleft = 0.8
time_text = ax.text(topleft, 0.95,'',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
cbar = fig.colorbar(sc,label='Initial depth [m]')
cbar.ax.invert_yaxis()

def animate(i,fig,sc):
    sc.set_offsets(np.c_[bdata['lon'][:,i], -bdata['z'][:,i]])
    ts = i*outputdt.microseconds/1000000
    if fb == 'b':
        ts = ts*-1
    time_text.set_text('time = %.1f seconds' % ts)
    return sc, time_text,

ax.set_xlabel("x [m]")
ax.set_ylabel("z (m)")
ax.set_xlim(-0.5,8.5)
ax.set_ylim(-0.49,0.5)
anim = animation.FuncAnimation(fig, animate, fargs = (fig, sc),
                               frames = len(bdata['lon'][0]), blit=True)
# anim.save('Figures/beaching.mp4')
plt.show()

Now we are going to look at the mass fluxes in each cell between 4 gridpoints to see if mass is conserved

In [7]:
# dy = float(flowdata['Y'][0]-flowdata['Y'][1])
# dx = float(flowdata['X'][1]-flowdata['X'][0])
# V = np.zeros((len(flowdata['T']),len(flowdata['X'])-1,len(flowdata['Y'])))
# U = np.zeros((len(flowdata['T']),len(flowdata['X']),len(flowdata['Y'])-1))
# massflux = np.zeros((len(flowdata['T']),len(flowdata['X'])-1,len(flowdata['Y'])-1))

In [15]:
# for i in range(len(flowdata['X'])-1):
#     V[:,i,:] = np.sum(flowdata['V'][:,:,i:i+1],axis=2)/2*dx
# print('done')
# for j in range(len(flowdata['Y'])-1):
#     U[:,:,j] = np.sum(flowdata['U'][:,j:j+1,:],axis=1)/2*dy
# print('done again')

# for i in range(len(flowdata['X'])-1):
#     for j in range(len(flowdata['Y'])-1):
#         massflux[:,i,j] = np.sum([U[:,i,j],-U[:,i+1,j],-V[:,i,j],V[:,i,j+1]],axis=0)
#     print(i)

# np.save('massfluxest0',massflux,allow_pickle=True)
massflux = np.load('massfluxest0.npy')

In [9]:
dy = float(flowdata['Y'][0]-flowdata['Y'][1])
dx = float(flowdata['X'][1]-flowdata['X'][0])
# U = np.zeros((len(flowdata['T']),len(flowdata['X'])-1,len(flowdata['Y'])-2))
# V = np.zeros((len(flowdata['T']),len(flowdata['X'])-2,len(flowdata['Y'])-1))
# massfluxb = np.zeros((len(flowdata['T']),len(flowdata['X'])-2,len(flowdata['Y'])-2))

# for i in range(len(flowdata['X'])-1):
#     U[:,i,:] = np.sum(flowdata['U'][:,1:-1,i:i+1],axis=2)/2*dx
# print('done')
# for j in range(len(flowdata['Y'])-1):
#     V[:,:,j] = np.sum(flowdata['V'][:,j:j+1,1:-1],axis=1)/2*dy
# print('done again')

# for i in range(len(flowdata['X'])-2):
#     for j in range(len(flowdata['Y'])-2):
#         massfluxb[:,i,j] = np.sum([U[:,i,j],-U[:,i+1,j],-V[:,i,j],V[:,i,j+1]],axis=0)
#     print(i)
    
# np.save('massfluxesbt0',massfluxb,allow_pickle=True)
massfluxb = np.load('massfluxesbt0.npy')

In [11]:
divergence = -massfluxb/dy/dx

In [12]:
divergence[0].shape

(898, 98)

Here we see that the divergence and convergence at the coral boundary achieves values of up to 40 per second. This means that at each second, the mass within a gridcell may increase or decrease 40 times.

In [14]:
%matplotlib qt
fig = plt.figure(figsize=(18,4))
ax = plt.axes()
# ax.plot(one['lon'].isel(obs=slice(tb-40,tb+1)),-one['z'].isel(obs=slice(tb-40,tb+1)),c='r',marker='o',label='particle trajectory')
pc = ax.pcolormesh(divergence[0].transpose(),vmin=-50,vmax=50,cmap='RdYlGn')
# pc = ax.pcolormesh(x,y,divergence[0].transpose(),vmin=-50,vmax=50,cmap='RdYlGn')
ax.invert_yaxis()
cbar = fig.colorbar(pc,label='Divergence [$1/s$]')
time_text = ax.text(0.05, 0.95,'time',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
ax.invert_yaxis()
ax.set_xlabel("x [m]")
ax.set_ylabel("height [m]")
plt.title('Divergence of the velocity field [1/s]')
def animate(i):
    pc.set_array(divergence[i].transpose().flatten())
    ts = i
    time_text.set_text('time = %.1f seconds' % ts)
#     return time_text,

# anim = animation.FuncAnimation(fig, animate,interval=1000, frames = len(flowdata['T']))
# anim.save('Figures/divergence1.mp4')

In [107]:
%matplotlib qt
fig = plt.figure(figsize=(18,4))
ax = plt.axes()
# ax.plot(one['lon'].isel(obs=slice(tb-40,tb+1)),-one['z'].isel(obs=slice(tb-40,tb+1)),c='r',marker='o',label='particle trajectory')
pc = ax.pcolormesh(x,y,massflux[0].transpose(),vmin=-np.max(abs(massflux)),vmax=np.max(abs(massflux)),cmap=cmocean.cm.balance)
cbar = fig.colorbar(pc,label='Flux [$m^2/s$]')
time_text = ax.text(0.05, 0.95,'time',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
ax.set_xlabel("x [m]")
ax.set_ylabel("z (m)")
plt.title('Cumulative flux into gridcells [$m^2/s$]')
def animate(i):
    pc.set_array(massflux[i].transpose().flatten())
    ts = i
    time_text.set_text('time = %.1f seconds' % ts)
#     return time_text,

anim = animation.FuncAnimation(fig, animate,interval=1000, frames = len(flowdata['T']))